In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers

In [30]:
# Read dataset
pdata = pd.read_csv("bank.csv")
pdata.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
pdata.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [32]:
# Drop the columns which are unique for all users
pdata = pdata.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)
pdata.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [34]:
pdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [35]:
# Convert categorical variables
pdata = pd.get_dummies(pdata, columns=['Geography'])
pdata = pd.get_dummies(pdata, columns=['Gender'])
pdata.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [36]:
# Distinguish the features and target variable
X =  pdata.drop(['Exited'], axis=1)
Y = pdata["Exited"]


In [37]:
# Normalize the train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 7)

X_train = preprocessing.normalize(X_train)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

(8000, 13)
(2000, 13)
(8000,)
(2000,)


In [38]:
# Initialize & build the model
model_classifier = Sequential()

# Add 2 hidden layers
model_classifier.add(Dense(16 ,activation='relu',input_dim = 13))
model_classifier.add(Dense(16,activation='tanh'))

# Add output layer
model_classifier.add(Dense(1, activation='sigmoid'))

In [42]:
model_classifier.compile(optimizer='sgd', loss='binary_crossentropy',  metrics = ['accuracy'])
model_classifier.summary()

# model is trained over 100 epochs
model_classifier.fit(X_train,Y_train,epochs=100,validation_data=(X_test,Y_test),batch_size=X_train.shape[0])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                224       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - 0s 135ms/step - loss: 0.7235 - accuracy: 0.4320 - val_loss: 0.7643 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 43ms/step - loss: 0.7199 - accuracy: 0.4378 - val_loss: 0.7620 - val_accuracy: 0.5015
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 0.7163 - accuracy: 0.4448 - val_l

In [43]:
Y_pred = model_classifier.predict(X_test)

In [45]:
results = model_classifier.evaluate(X_test, Y_test.values)

63/63 [==============================] - 0s 1ms/step - loss: 0.6253 - accuracy: 0.7770


In [46]:
print(model_classifier.metrics_names)
print(results)  

['loss', 'accuracy']
[0.6253044009208679, 0.7770000100135803]


In [47]:
# Confusion Matrix
Y_pred_cls = model_classifier.predict_classes(X_test, batch_size=200, verbose=0)
print('Accuracy Model1 (Dropout): '+ str(model_classifier.evaluate(X_test,Y_test.values)[1]))
print('Recall_score: ' + str(recall_score(Y_test.values,Y_pred_cls)))
print('Precision_score: ' + str(precision_score(Y_test.values, Y_pred_cls)))
print('F-score: ' + str(f1_score(Y_test.values,Y_pred_cls)))
confusion_matrix(Y_test.values, Y_pred_cls)

63/63 [==============================] - 0s 1ms/step - loss: 0.6253 - accuracy: 0.7770
Accuracy Model1 (Dropout): 0.7770000100135803
Recall_score: 0.014598540145985401
Precision_score: 0.1276595744680851
F-score: 0.026200873362445413


array([[1548,   41],
       [ 405,    6]])